<a href="https://colab.research.google.com/github/chachak410/ai-design-generator/blob/main/website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile index.html
<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>AI Design Generator (Browser-only)</title>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <style>
    body { font-family: system-ui, sans-serif; margin: 20px; color: #222; }
    .row { display: flex; gap: 12px; flex-wrap: wrap; margin-bottom: 12px; }
    textarea, input, select, button { padding: 8px; font-size: 14px; }
    textarea { width: 100%; height: 140px; }
    .controls { max-width: 900px; }
    .grid { display: grid; grid-template-columns: repeat(auto-fill, minmax(220px, 1fr)); gap: 12px; }
    .card { border: 1px solid #ddd; border-radius: 8px; padding: 8px; }
    .card img { width: 100%; height: auto; display: block; border-radius: 4px; }
    .hint { color: #666; font-size: 12px; }
    .badge { background:#eee; padding:2px 6px; border-radius: 4px; font-size: 12px; }
    #status { margin: 8px 0; }
  </style>
</head>
<body>
  <h1>AI Design Generator</h1>
  <div class="controls">
    <div class="row">
      <textarea id="prompt" placeholder="Describe your design (e.g., 'Minimalist product hero shot, studio lighting, soft shadows, high detail')"></textarea>
    </div>
    <div class="row">
      <label>Provider:
        <select id="provider">
          <option value="auto" selected>Auto (Horde -> Pollinations)</option>
          <option value="pollinations">Pollinations only</option>
          <option value="stable_horde">Stable Horde only</option>
          <option value="one_each">One from each</option>
        </select>
      </label>
      <label>Count:
        <input id="count" type="number" min="1" max="8" value="3"/>
      </label>
      <label>Aspect ratio:
        <select id="aspect">
          <option>1:1</option>
          <option selected>3:4</option>
          <option>4:3</option>
          <option>9:16</option>
          <option>16:9</option>
        </select>
      </label>
      <label>Seed (optional):
        <input id="seed" type="number" placeholder="e.g. 12345"/>
      </label>
    </div>
    <div class="row">
      <label>Horde API key (optional):
        <input id="hordeKey" type="text" placeholder="leave empty for anonymous"/>
      </label>
      <label>Horde Proxy URL (optional):
        <input id="hordeProxy" type="text" placeholder="https://your-worker.workers.dev"/>
      </label>
      <button id="go">Generate</button>
    </div>
    <div id="status" class="hint"></div>
  </div>

  <div id="results" class="grid"></div>

  <script>
    const el = (id) => document.getElementById(id);

    function aspectToSize(ar) {
      const map = { "1:1": [768,768], "3:4": [768,1024], "4:3": [1024,768], "9:16": [768,1365], "16:9": [1365,768] };
      return map[ar] || [768,1024];
    }

    function pollinationsUrls(prompt, count, width, height, seed) {
      const quoted = encodeURIComponent(prompt);
      const base = `https://image.pollinations.ai/prompt/${quoted}?width=${width}&height=${height}&n=1`;
      const startSeed = Number.isInteger(seed) ? seed : Date.now();
      return Array.from({length: count}, (_, i) => `${base}&seed=${startSeed + i}`);
    }

    async function hordeSubmit(prompt, width, height, seed, apiKey, proxyBase) {
      const payload = {
        prompt,
        params: {
          sampler_name: "k_euler",
          cfg_scale: 7,
          seed: Number.isInteger(seed) ? seed : null,
          steps: 28,
          width: Math.round(width / 64) * 64,
          height: Math.round(height / 64) * 64
        },
        nsfw: false,
        censor_nsfw: true,
        trusted_workers: false,
        slow_workers: true
      };
      const base = proxyBase || "https://stablehorde.net/api/v2";
      const r = await fetch(base + "/generate/async", {
        method: "POST",
        headers: {
          "Content-Type": "application/json",
          "accept": "application/json",
          "apikey": apiKey || "0000000000"
        },
        body: JSON.stringify(payload),
        mode: "cors"
      });
      if (!r.ok) throw new Error("Horde submit failed: " + r.status);
      const j = await r.json();
      if (!j.id) throw new Error("No job id from Horde");
      return j.id;
    }

    async function hordePoll(id, apiKey, proxyBase, timeoutMs = 240000, pollMs = 4000) {
      const start = Date.now();
      const base = proxyBase || "https://stablehorde.net/api/v2";
      while (Date.now() - start < timeoutMs) {
        const r = await fetch(`${base}/generate/status/${id}`, {
          headers: { "accept": "application/json", "apikey": apiKey || "0000000000" },
          mode: "cors",
        });
        if (!r.ok) throw new Error("Horde status failed: " + r.status);
        const j = await r.json();
        if (j.done && j.generations && j.generations.length) return j;
        await new Promise(res => setTimeout(res, pollMs));
      }
      throw new Error("Horde timed out");
    }

    function render(items) {
      const container = el("results");
      container.innerHTML = "";
      items.forEach((it, idx) => {
        const card = document.createElement("div");
        card.className = "card";
        const src = it.url || (it.b64 ? ("data:image/png;base64," + it.b64) : "");
        card.innerHTML = `
          <div class="badge">${it.provider || "unknown"}</div>
          ${src ? `<img src="${src}" alt="img-${idx}"/>` : "<div>No image</div>"}
        `;
        container.appendChild(card);
      });
    }

    async function generate() {
      const prompt = el("prompt").value.trim();
      const count = parseInt(el("count").value || "1", 10);
      const aspect = el("aspect").value;
      const seedRaw = el("seed").value.trim();
      const seed = seedRaw ? parseInt(seedRaw, 10) : null;
      const provider = el("provider").value;
      const hordeKey = el("hordeKey").value.trim() || null;
      const proxy = el("hordeProxy").value.trim() || null;

      if (!prompt) {
        el("status").textContent = "Please enter a prompt.";
        return;
      }
      const [w, h] = aspectToSize(aspect);
      el("status").textContent = "Working...";
      el("go").disabled = true;

      try {
        let items = [];
        if (provider === "pollinations") {
          items = pollinationsUrls(prompt, count, w, h, seed).map(u => ({ provider: "pollinations", url: u }));
        } else if (provider === "stable_horde") {
          for (let i = 0; i < count; i++) {
            const id = await hordeSubmit(prompt, w, h, Number.isInteger(seed) ? seed + i : null, hordeKey, proxy);
            const status = await hordePoll(id, hordeKey, proxy);
            const gen = status.generations.find(g => g.img);
            if (gen && gen.img) items.push({ provider: "stable_horde", b64: gen.img });
          }
        } else if (provider === "one_each") {
          try {
            const id = await hordeSubmit(prompt, w, h, seed, hordeKey, proxy);
            const status = await hordePoll(id, hordeKey, proxy);
            const gen = status.generations.find(g => g.img);
            if (gen && gen.img) items.push({ provider: "stable_horde", b64: gen.img });
          } catch (e) {
            console.warn("Horde failed:", e);
          }
          items.push({ provider: "pollinations", url: pollinationsUrls(prompt, 1, w, h, seed)[0] });
          while (items.length < count) {
            const s = Number.isInteger(seed) ? seed + items.length : null;
            items.push({ provider: "pollinations", url: pollinationsUrls(prompt, 1, w, h, s)[0] });
          }
        } else {
          // Auto: try Horde then fallback to Pollinations
          try {
            for (let i = 0; i < count; i++) {
              const id = await hordeSubmit(prompt, w, h, Number.isInteger(seed) ? seed + i : null, hordeKey, proxy);
              const status = await hordePoll(id, hordeKey, proxy);
              const gen = status.generations.find(g => g.img);
              if (gen && gen.img) items.push({ provider: "stable_horde", b64: gen.img });
            }
          } catch (e) {
            console.warn("Horde failed, using Pollinations:", e);
          }
          if (items.length < count) {
            const needed = count - items.length;
            const start = Number.isInteger(seed) ? seed + items.length : null;
            const urls = pollinationsUrls(prompt, needed, w, h, start);
            items.push(...urls.map(u => ({ provider: "pollinations", url: u })));
          }
        }

        render(items);
        el("status").textContent = `Done. Returned ${items.length} image(s).`;
      } catch (e) {
        console.error(e);
        el("status").textContent = "Error: " + (e.message || e);
      } finally {
        el("go").disabled = false;
      }
    }

    document.getElementById("go").addEventListener("click", generate);
  </script>
</body>
</html>

Writing index.html


In [2]:
from IPython.display import HTML
HTML(open('index.html', 'r').read())

In [3]:
from google.colab import files
files.download('index.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>